# Import libraries

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Import dataset

In [7]:
dataset = pd.read_csv('master.csv')
X = dataset
y = dataset.pop('suicides/100k pop')

# Take care of missing data

In [23]:
X[' gdp_for_year ($) '] = X[' gdp_for_year ($) '].str.replace(',', '').astype(float)

In [29]:
X_num = X.select_dtypes(include=['int64', 'float64']).copy()
X_cat = X.select_dtypes(include=['object']).copy()

In [36]:
from sklearn.preprocessing import Imputer
num_imputer = Imputer(missing_values='NaN', strategy='mean')
num_imputer = num_imputer.fit(X_num)
#X_num[:, :] = num_imputer(X_num[:, :])

C:\Users\Justin\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [33]:
X_num.isnull().sum().sum()

AttributeError: 'numpy.ndarray' object has no attribute 'isnull'